In [22]:
# pytorch
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import numpy as np
import random

# pytorch lightning
from lightning import LightningModule
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from torchmetrics.classification import MultilabelF1Score

import pandas as pd
import utils

In [26]:
SEED = 42 #choose a seed

def fix_seed(seed: int) -> None:
    
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_seed(SEED)

In [27]:

df_arg_train = pd.read_csv('./data/arguments-training.tsv', sep='\t')
df_arg_test = pd.read_csv('./data/arguments-test.tsv', sep='\t')
df_arg_val = pd.read_csv('./data/arguments-validation.tsv', sep='\t')

df_labels_train = pd.read_csv('./data/labels-training.tsv', sep='\t')
df_labels_test = pd.read_csv('./data/labels-test.tsv', sep='\t')
df_labels_val = pd.read_csv('./data/labels-validation.tsv', sep='\t')

df_labels_test.head()

,Argument ID,Self-direction: thought,Self-direction: action,Stimulation,Hedonism,Achievement,Power: dominance,Power: resources,Face,Security: personal,...,Tradition,Conformity: rules,Conformity: interpersonal,Humility,Benevolence: caring,Benevolence: dependability,Universalism: concern,Universalism: nature,Universalism: tolerance,Universalism: objectivity
0,A26004,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,1,0,1,0
1,A26010,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
2,A26016,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,1,1,0,0,0
3,A26024,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A26026,0,0,0,0,1,0,0,0,1,...,0,0,0,0,1,1,0,0,0,0


In [28]:
level_3_categories = ["Openness to change", "Self-enhancement", "Conservation", "Self-transcendence"]

level_3_to_2_mapping = {
    "Openness to change": [
        "Self-direction: thought",
        "Self-direction: action",
        "Stimulation",
        "Hedonism",
    ],
    "Self-enhancement": [
        "Hedonism",
        "Achievement",
        "Power: dominance",
        "Power: resources",
        "Face",
    ],
    "Conservation": [
        "Security: personal",
        "Security: societal",
        "Conformity: rules",
        "Conformity: interpersonal",
        "Tradition",
        "Face",
        "Humility",
    ],
    "Self-transcendence": [
        "Benevolence: caring",
        "Benevolence: dependability",
        "Universalism: concern",
        "Universalism: nature",
        "Universalism: tolerance",
        "Universalism: objectivity",
        "Humility",
    ]
}

column_to_drop = [x for l in level_3_to_2_mapping.values() for x in l]

for category in level_3_categories:
    # make a logical OR of all the level 2 categories
    df_labels_test[category] = df_labels_test[level_3_to_2_mapping[category]].any(axis=1).map({True: 1, False: 0})
    df_labels_val[category] = df_labels_val[level_3_to_2_mapping[category]].any(axis=1).map({True: 1, False: 0})
    df_labels_train[category] = df_labels_train[level_3_to_2_mapping[category]].any(axis=1).map({True: 1, False: 0})

df_labels_test = df_labels_test.drop(columns=column_to_drop)
df_labels_val = df_labels_val.drop(columns=column_to_drop)
df_labels_train = df_labels_train.drop(columns=column_to_drop)

df_labels_test.head()

,Argument ID,Openness to change,Self-enhancement,Conservation,Self-transcendence
0,A26004,0,1,1,1
1,A26010,0,1,0,1
2,A26016,0,1,1,1
3,A26024,0,1,0,0
4,A26026,0,1,1,1


In [29]:
df_train = pd.merge(df_arg_train, df_labels_train, on='Argument ID')
df_test = pd.merge(df_arg_test, df_labels_test, on='Argument ID')
df_val = pd.merge(df_arg_val, df_labels_val, on='Argument ID')

df_train.head()

,Argument ID,Conclusion,Stance,Premise,Openness to change,Self-enhancement,Conservation,Self-transcendence
0,A01002,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...,0,0,1,0
1,A01005,We should ban fast food,in favor of,fast food should be banned because it is reall...,0,0,1,0
2,A01006,We should end the use of economic sanctions,against,sometimes economic sanctions are the only thin...,0,1,1,0
3,A01007,We should abolish capital punishment,against,capital punishment is sometimes the only optio...,0,0,1,1
4,A01008,We should ban factory farming,against,factory farming allows for the production of c...,0,0,1,1


### Task 1.5 Encoding

In [30]:
# Encode stance into 0, 1 

df_train["Stance"] = df_train["Stance"].map({"in favor of": 1, "against": 0})
df_test["Stance"] = df_test["Stance"].map({"in favor of": 1, "against": 0})
df_val["Stance"] = df_val["Stance"].map({"in favor of": 1, "against": 0})

df_train.head()

,Argument ID,Conclusion,Stance,Premise,Openness to change,Self-enhancement,Conservation,Self-transcendence
0,A01002,We should ban human cloning,1,we should ban human cloning as it will only ca...,0,0,1,0
1,A01005,We should ban fast food,1,fast food should be banned because it is reall...,0,0,1,0
2,A01006,We should end the use of economic sanctions,0,sometimes economic sanctions are the only thin...,0,1,1,0
3,A01007,We should abolish capital punishment,0,capital punishment is sometimes the only optio...,0,0,1,1
4,A01008,We should ban factory farming,0,factory farming allows for the production of c...,0,0,1,1


## Dataset definition

In [7]:
class ArgumentDataset(Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        return {
            "Premise": row["Premise"],
            "Conclusion": row["Conclusion"],
            "labels": torch.tensor(row[level_3_categories].values.tolist(), dtype=torch.float32),
            "stance": torch.tensor(row["Stance"], dtype=torch.float32)
        }

In [33]:
train_dataset = ArgumentDataset(df_train)
test_dataset = ArgumentDataset(df_test)
val_dataset = ArgumentDataset(df_val)
# Create the dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(train_dataset[0])

{'Premise': 'we should ban human cloning as it will only cause huge issues when you have a bunch of the same humans running around all acting the same.', 'Conclusion': 'We should ban human cloning', 'labels': tensor([0., 0., 1., 0.]), 'stance': tensor(1.)}


## Task 3 Metric definition

## Task 2 Model definition

### Random and Majority Classifier

In [34]:
class RandomUniformClassifier(LightningModule):
    def __init__(self):
        self._random_state = np.random.RandomState()

    def predict(self, X):
        batch_size = X.shape[0]
        logits = self._random_state.uniform(size=(batch_size, 4))
        logits = logits > 0.5
        return torch.tensor(logits, dtype=torch.float32)


class MajorityClassifier(LightningModule):
    def __init__(self, n_random_classifiers=10):
        self.n_random_classifiers = n_random_classifiers
        self.random_classifiers = [RandomUniformClassifier() for _ in range(n_random_classifiers)]

    def predict(self, X):
        batch_size = X.shape[0]
        votes = torch.zeros((batch_size, 4))
        for clf in self.random_classifiers:
            votes += clf.predict(X)
        votes = votes / self.n_random_classifiers
        votes = votes > 0.5
        return torch.tensor(votes, dtype=torch.float32)

### Bert models

In [10]:
from transformers import BertModel, BertTokenizer


class BertConclusion(LightningModule):
    def __init__(self, bert_model_name, num_classes):
        super().__init__()
        self.save_hyperparameters()

        self.tokenizer = BertTokenizer.from_pretrained(bert_model_name)
        self.bert = BertModel.from_pretrained(bert_model_name)
        for param in self.bert.parameters():
            param.requires_grad = False

        self.classifier = nn.Linear(self.bert.config.hidden_size, num_classes)

        self.f1_metric = MultilabelF1Score(4, average=None)

    def forward(self, encoded):
        outputs = self.bert(**encoded)
        logits = self.classifier(outputs.last_hidden_state[:, 0, :])
        return logits

    def training_step(self, batch, batch_idx):
        data = batch
        X = data["Conclusion"]
        y = data["labels"]

        encoded = self.tokenizer(X, padding=True, truncation=True, return_tensors="pt")
        logits = self(encoded)

        loss = nn.BCEWithLogitsLoss()(logits, y)
        self.log("train_loss", loss, on_epoch=True, prog_bar=True, logger=True)

        f1_score_per_class = self.f1_metric(logits, y)
        f1_score_mean = torch.mean(f1_score_per_class)

        self.log("train_f1_score", f1_score_mean, on_epoch=True, prog_bar=True, logger=True)

        for i, category in enumerate(level_3_categories):
            self.log(f"train_f1_score_{category}", f1_score_per_class[i], on_epoch=True, prog_bar=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        data = batch
        X = data["Conclusion"]
        y = data["labels"]

        encoded = self.tokenizer(X, padding=True, truncation=True, return_tensors="pt")
        logits = self(encoded)

        loss = nn.BCEWithLogitsLoss()(logits, y)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True)

        f1_score_per_class = self.f1_metric(logits, y)
        f1_score_mean = torch.mean(f1_score_per_class)

        self.log("val_f1_score", f1_score_mean, on_epoch=True, prog_bar=True, logger=True)

        for i, category in enumerate(level_3_categories):
            self.log(f"val_f1_score_{category}", f1_score_per_class[i], on_epoch=True, prog_bar=True, logger=True)

        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-5)

In [11]:
class BertPremiseConclusion(LightningModule):
    def __init__(self, bert_model_name, num_classes):
        super().__init__()
        self.save_hyperparameters()

        self.tokenizer = BertTokenizer.from_pretrained(bert_model_name)
        self.bert = BertModel.from_pretrained(bert_model_name)
        for param in self.bert.parameters():
            param.requires_grad = False

        self.classifier = nn.Linear(self.bert.config.hidden_size * 2, num_classes)

        self.f1_metric = MultilabelF1Score(4, average=None)

    def forward(self, encoded_1, encoded_2):
        output_1 = self.bert(**encoded_1)
        output_2 = self.bert(**encoded_2)

        output = torch.cat((output_1.last_hidden_state[:, 0, :], output_2.last_hidden_state[:, 0, :]), dim=1)

        logits = self.classifier(output)
        return logits

    def training_step(self, batch, batch_idx):
        data = batch

        X_1, X_2 = data["Conclusion"], data["Conclusion"]
        y = data["labels"]

        encoded_1 = self.tokenizer(X_1, padding=True, truncation=True, return_tensors="pt")
        encoded_2 = self.tokenizer(X_2, padding=True, truncation=True, return_tensors="pt")

        logits = self(encoded_1, encoded_2)

        loss = nn.BCEWithLogitsLoss()(logits, y)
        self.log("train_loss", loss, on_epoch=True, prog_bar=True, logger=True)

        f1_score_per_class = self.f1_metric(logits, y)
        f1_score_mean = torch.mean(f1_score_per_class)

        self.log("train_f1_score", f1_score_mean, on_epoch=True, prog_bar=True, logger=True)

        for i, category in enumerate(level_3_categories):
            self.log(f"train_f1_score_{category}", f1_score_per_class[i], on_epoch=True, prog_bar=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        data = batch

        X_1, X_2 = data["Conclusion"], data["Conclusion"]
        y = data["labels"]

        encoded_1 = self.tokenizer(X_1, padding=True, truncation=True, return_tensors="pt")
        encoded_2 = self.tokenizer(X_2, padding=True, truncation=True, return_tensors="pt")

        logits = self(encoded_1, encoded_2)

        loss = nn.BCEWithLogitsLoss()(logits, y)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True)

        f1_score_per_class = self.f1_metric(logits, y)
        f1_score_mean = torch.mean(f1_score_per_class)

        self.log("val_f1_score", f1_score_mean, on_epoch=True, prog_bar=True, logger=True)

        for i, category in enumerate(level_3_categories):
            self.log(f"val_f1_score_{category}", f1_score_per_class[i], on_epoch=True, prog_bar=True, logger=True)

        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-5)

In [12]:
class BertPremiseConclusionStance(LightningModule):
    def __init__(self, bert_model_name, num_classes):
        super().__init__()
        self.save_hyperparameters()

        self.tokenizer = BertTokenizer.from_pretrained(bert_model_name)
        self.bert = BertModel.from_pretrained(bert_model_name)
        for param in self.bert.parameters():
            param.requires_grad = False

        self.classifier = nn.Linear(self.bert.config.hidden_size * 3, num_classes)

        self.f1_metric = MultilabelF1Score(4, average=None)

    def forward(self, encoded_1, encoded_2, encoded_3):
        output_1 = self.bert(**encoded_1).last_hidden_state[:, 0, :]
        output_2 = self.bert(**encoded_2).last_hidden_state[:, 0, :]
        output_3 = self.bert(**encoded_3).last_hidden_state[:, 0, :]

        output = torch.cat((output_1, output_2, output_3), dim=1)

        logits = self.classifier(output)
        return logits

    def training_step(self, batch, batch_idx):
        data = batch

        X_1, X_2, X_3 = data["Premise"], data["Conclusion"], data["Conclusion"]
        y = data["labels"]

        encoded_1 = self.tokenizer(X_1, padding=True, truncation=True, return_tensors="pt")
        encoded_2 = self.tokenizer(X_2, padding=True, truncation=True, return_tensors="pt")
        encoded_3 = self.tokenizer(X_3, padding=True, truncation=True, return_tensors="pt")

        logits = self(encoded_1, encoded_2, encoded_3)

        loss = nn.BCEWithLogitsLoss()(logits, y)
        self.log("train_loss", loss, on_epoch=True, prog_bar=True, logger=True)

        f1_score_per_class = self.f1_metric(logits, y)
        f1_score_mean = torch.mean(f1_score_per_class)

        self.log("train_f1_score", f1_score_mean, on_epoch=True, prog_bar=True, logger=True)

        for i, category in enumerate(level_3_categories):
            self.log(f"train_f1_score_{category}", f1_score_per_class[i], on_epoch=True, prog_bar=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        data = batch

        X_1, X_2, X_3 = data["Premise"], data["Conclusion"], data["Conclusion"]
        y = data["labels"]

        encoded_1 = self.tokenizer(X_1, padding=True, truncation=True, return_tensors="pt")
        encoded_2 = self.tokenizer(X_2, padding=True, truncation=True, return_tensors="pt")
        encoded_3 = self.tokenizer(X_3, padding=True, truncation=True, return_tensors="pt")

        logits = self(encoded_1, encoded_2, encoded_3)

        loss = nn.BCEWithLogitsLoss()(logits, y)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True)

        f1_score_per_class = self.f1_metric(logits, y)
        f1_score_mean = torch.mean(f1_score_per_class)

        self.log("val_f1_score", f1_score_mean, on_epoch=True, prog_bar=True, logger=True)

        for i, category in enumerate(level_3_categories):
            self.log(f"val_f1_score_{category}", f1_score_per_class[i], on_epoch=True, prog_bar=True, logger=True)

        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-5)

Trainer definition

In [13]:
trainer = Trainer(
    max_epochs=10,
    logger=TensorBoardLogger("lightning_logs", name="bert"),
    callbacks=[
        ModelCheckpoint(monitor="val_loss"),
        EarlyStopping(monitor="val_loss", patience=3),
    ],
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


F1 metric definition

In [14]:
f1_metric = MultilabelF1Score(num_labels=4, average=None, multidim_average='global')

Train all the models on the train set and evaluate them:

1) BertConclusion Model: training, predictions, f1 score

In [38]:
bertConclusion= BertConclusion("bert-base-uncased", num_classes=len(level_3_categories))

trainer.fit(bertConclusion, train_dataloader, val_dataloader)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:630: Checkpoint directory lightning_logs\bert\version_2\checkpoints e

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 6: 100%|██████████| 169/169 [01:30<00:00,  1.87it/s, v_num=2, train_loss_step=0.657, train_f1_score_step=0.403, train_f1_score_Openness to change_step=0.000, train_f1_score_Self-enhancement_step=0.000, train_f1_score_Conservation_step=0.828, train_f1_score_Self-transcendence_step=0.786, val_loss=0.621, val_f1_score=0.459, val_f1_score_Openness to change=0.000, val_f1_score_Self-enhancement=0.100, val_f1_score_Conservation=0.854, val_f1_score_Self-transcendence=0.882, train_loss_epoch=0.639, train_f1_score_epoch=0.524, train_f1_score_Openness to change_epoch=0.0306, train_f1_score_Self-enhancement_epoch=0.344, train_f1_score_Conservation_epoch=0.860, train_f1_score_Self-transcendence_epoch=0.862]


In [39]:
predictions_bertConclusion = utils.model_predict_1(bertConclusion, test_dataloader)
print(predictions_bertConclusion.shape)

torch.Size([1576, 4])


In [40]:
selected_columns = ['Openness to change', 'Self-enhancement', 'Conservation', 'Self-transcendence']
df_labels_test = df_labels_test[selected_columns]
target_tensor = torch.tensor(df_labels_test.values)

In [42]:
results_bConclusion = f1_metric(predictions_bertConclusion, target_tensor)
average_bConclusion = sum(results_bConclusion)/4
print("F1 Score for each class:")
for i, category in enumerate(level_3_categories):
    print(f"{category}: {results_bConclusion[i]:.4f}")

print(f"Overall F1 Score: {average_bConclusion:.4f}")

F1 Score for each class:
Openness to change: 0.0084
Self-enhancement: 0.4011
Conservation: 0.8304
Self-transcendence: 0.8917
Overall F1 Score: 0.5329


2) BertPremiseConclusion Model: training, predictions, f1 score

In [ ]:
bertPremiseConclusion= BertPremiseConclusion("bert-base-uncased", num_classes=len(level_3_categories))

trainer.fit(bertPremiseConclusion, train_dataloader, val_dataloader)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

  | Name       | Type              | Params
-------------------------------------------------
0 | bert       | BertModel         | 109 M 
1 | classifier | Linear            | 6.1 K 
2 | f1

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 9: 100%|██████████| 169/169 [02:13<00:00,  1.26it/s, v_num=1, train_loss_step=0.663, train_f1_score_step=0.464, train_f1_score_Openness to change_step=0.000, train_f1_score_Self-enhancement_step=0.333, train_f1_score_Conservation_step=0.938, train_f1_score_Self-transcendence_step=0.583, val_loss=0.600, val_f1_score=0.553, val_f1_score_Openness to change=0.000, val_f1_score_Self-enhancement=0.475, val_f1_score_Conservation=0.854, val_f1_score_Self-transcendence=0.882, train_loss_epoch=0.609, train_f1_score_epoch=0.511, train_f1_score_Openness to change_epoch=0.000848, train_f1_score_Self-enhancement_epoch=0.320, train_f1_score_Conservation_epoch=0.862, train_f1_score_Self-transcendence_epoch=0.861]  

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 169/169 [02:16<00:00,  1.24it/s, v_num=1, train_loss_step=0.663, train_f1_score_step=0.464, train_f1_score_Openness to change_step=0.000, train_f1_score_Self-enhancement_step=0.333, train_f1_score_Conservation_step=0.938, train_f1_score_Self-transcendence_step=0.583, val_loss=0.600, val_f1_score=0.553, val_f1_score_Openness to change=0.000, val_f1_score_Self-enhancement=0.475, val_f1_score_Conservation=0.854, val_f1_score_Self-transcendence=0.882, train_loss_epoch=0.609, train_f1_score_epoch=0.511, train_f1_score_Openness to change_epoch=0.000848, train_f1_score_Self-enhancement_epoch=0.320, train_f1_score_Conservation_epoch=0.862, train_f1_score_Self-transcendence_epoch=0.861]


In [ ]:
predictions_bertPremiseConclusion = utils.model_predict_2(bertPremiseConclusion, test_dataloader)
print(predictions_bertPremiseConclusion.shape)

torch.Size([1576, 4])


In [ ]:
results_bPC = f1_metric(predictions_bertPremiseConclusion, target_tensor)
average_bPC = sum(results_bPC)/4
print("F1 Score for each class:")
for i, category in enumerate(level_3_categories):
    print(f"{category}: {results_bPC[i]:.4f}")

print(f"Overall F1 Score: {average_bPC:.4f}")

F1 Score for each class:
Openness to change: 0.0000
Self-enhancement: 0.2747
Conservation: 0.8304
Self-transcendence: 0.8917
Overall F1 Score: 0.4992


3) BertPremiseConclusionStance Model: training, predictions, f1 metric

In [15]:
bertClassifier = BertPremiseConclusionStance("bert-base-uncased", num_classes=len(level_3_categories))

trainer.fit(bertClassifier, train_dataloader, val_dataloader)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

  | Name       | Type              | Params
-------------------------------------------------
0 | bert       | BertModel         | 109 M 
1 | classifier | Linear            | 9.2 K 
2 | f1

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Epoch 4:  73%|███████▎  | 123/169 [04:37<01:43,  0.44it/s, v_num=2, train_loss_step=0.596, train_f1_score_step=0.491, train_f1_score_Openness to change_step=0.000, train_f1_score_Self-enhancement_step=0.250, train_f1_score_Conservation_step=0.877, train_f1_score_Self-transcendence_step=0.836, val_loss=0.596, val_f1_score=0.516, val_f1_score_Openness to change=0.000, val_f1_score_Self-enhancement=0.328, val_f1_score_Conservation=0.854, val_f1_score_Self-transcendence=0.882, train_loss_epoch=0.606, train_f1_score_epoch=0.503, train_f1_score_Openness to change_epoch=0.000989, train_f1_score_Self-enhancement_epoch=0.288, train_f1_score_Conservation_epoch=0.863, train_f1_score_Self-transcendence_epoch=0.862] 

c:\Users\Acer\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [16]:
predictions_bertClassifier = utils.model_predict_3(bertClassifier, test_dataloader)
print(predictions_bertClassifier.shape)


torch.Size([1576, 4])


In [ ]:

results_bC = f1_metric(predictions_bertClassifier, target_tensor)
average_bC = sum(results_bC)/4
print("F1 Score for each class:")
for i, category in enumerate(level_3_categories):
    print(f"{category}: {results_bC[i]:.4f}")

print(f"Overall F1 Score: {average_bC:.4f}")


F1 Score for each class:
Openness to change: 0.0000
Self-enhancement: 0.3473
Conservation: 0.8304
Self-transcendence: 0.8917
Overall F1 Score: 0.5174
